In [1]:
import os
import csv
import schedule
import time
import pandas as pd
from csv import writer
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Set API parameters.
load_dotenv()
API_KEY = os.getenv("API_KEY")
CHANNEL_ID = os.getenv("CHANNEL_ID")
youtube = build('youtube','v3', developerKey = API_KEY)

class ytStats:
    def __init__(self, API_KEY, CHANNEL_ID, youtube):
        self.API_KEY = API_KEY
        self.CHANNEL_ID = CHANNEL_ID
        self.youtube = youtube
    
    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = CHANNEL_ID
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title']  , 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'] ,
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount']  , 
                result_channel_views = response['items'][i]['statistics']['viewCount']  , 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    def get_video_ids(self):
        playlist_id = ytStats.get_channel_statistics(youtube)[0]['result_playlist_id']
        
        request = youtube.playlistItems().list(
            part = 'contentDetails' , 
            playlistId = playlist_id , 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails' , 
                    playlistId = playlist_id , 
                    maxResults = 50 , 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    def get_video_details(self):
        video_ids = ytStats.get_video_ids(youtube)

        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics' , 
                id = ','.join(video_ids[i: i + 50]) ,
                #order = 'date' # DOES NOT WORK
            )
            response = request.execute()

            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title']  ,
                    result_video_id = video['id'] ,                
                    result_video_description = video['snippet']['description']  ,
                    result_video_upload_time = video['snippet']['publishedAt']  ,
                    result_video_views = video['statistics']['viewCount'] ,  
                    result_video_likes = video['statistics']['likeCount'] ,  
                    result_video_favorites = video['statistics']['favoriteCount'] ,  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)

        return all_data

In [15]:
# Call functions
yt = ytStats
channel_stats, video_details = yt.get_channel_statistics(youtube), yt.get_video_details(youtube)

In [16]:
#class ytOutput:
def create_data_frame(channel_stats, video_details):
    # Create data frames.
    initial_channel_data = pd.DataFrame(channel_stats, columns = [
        'result_playlist_id' ,
        'result_channel_name' , 
        'result_channel_views' ,
        'result_channel_subscribers' ,
        'result_channel_total_videos' 
        ])
    initial_video_data = pd.DataFrame(video_details, columns = [
        'result_video_id' ,
        'result_video_upload_time' , 
        'result_video_name' , 
        'result_video_description' , 
        'result_video_views' , 
        'result_video_likes' , 
        'result_video_comments' 
        ])
    sort = initial_video_data.sort_values('result_video_upload_time', axis = 0, ascending = False)

    # Organize column output. 
    # video_data = initial_video_data.reindex(columns = ['result_video_id', 'published_date', 'published_time', 'result_video_name','result_video_description','result_video_views','result_video_likes','result_video_comments'])
    # video_data.columns = ['Video ID', 'Date', 'Time', 'Video Name', 'Video Description', 'Views', 'Likes', 'Comments', ]
    return initial_channel_data, initial_video_data, sort

channel_data, video_data, sort = create_data_frame(channel_stats, video_details)
#sort = pd.DataFrame.sort_values(by = 'result_video_upload_time', axis=0, ascending=True, kind='mergesort')

In [17]:
with open('temp_data.csv', 'w') as csvfile:
    fieldnames = ['Video ID']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    writer.writerow({"Video ID":vid_id})